.

Data_Engineering_TIL(20200718)


Study Program : Docker 기반의 DevOps 인프라 구축 WORKSHOP


관련 URL : https://www.fastcampus.co.kr/dev_workshop_devops


학습자료 출처 : http://www.authorstream.com/Presentation/k16wire-4227985-devops-docker-v7/

[학습내용]

#### 1. 도커 이미지 백업

- 도커이미지를 파일로 저장하고, 로드도 할 수 있다.


- 네트워크에서 단절된 곳에서 활용하면 유용한 방법이다.


- 장점 : 인터넷 연결이 안되어서 이미지 다운이 안되는 경우


- 단점 : 이미지 크기에 따라 저장 및 로딩에 시간이 오래걸린다.


아래 리눅스 커맨드 명령어와 같이 도커 이미지를 백업하고, 로딩할 수 있다.

In [ ]:
[root@ip-10-1-10-245 ec2-user]# docker ps
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES

[root@ip-10-1-10-245 ec2-user]# docker pull busybox
Using default tag: latest
latest: Pulling from library/busybox
91f30d776fb2: Pull complete
Digest: sha256:9ddee63a712cea977267342e8750ecbc60d3aab25f04ceacfa795e6fce341793
Status: Downloaded newer image for busybox:latest
docker.io/library/busybox:latest

[root@ip-10-1-10-245 ec2-user]# docker images
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
busybox             latest              c7c37e472d31        2 weeks ago         1.22MB

busybox 이미지를 압축파일(tar) 파일로 복사
[root@ip-10-1-10-245 ec2-user]# docker save busybox > busybox.tar

[root@ip-10-1-10-245 ec2-user]# ls
busybox.tar

[root@ip-10-1-10-245 ec2-user]# docker images
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
busybox             latest              c7c37e472d31        2 weeks ago         1.22MB

[root@ip-10-1-10-245 ec2-user]# docker rmi busybox
Untagged: busybox:latest
Untagged: busybox@sha256:9ddee63a712cea977267342e8750ecbc60d3aab25f04ceacfa795e6fce341793
Deleted: sha256:c7c37e472d31c1685b48f7004fd6a64361c95965587a951692c5f298c6685998
Deleted: sha256:50761fe126b6e4d90fa0b7a6e195f6030fe250c016c2fc860ac40f2e8d2f2615

[root@ip-10-1-10-245 ec2-user]# docker images
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE

busybox.jar 파일을 이미지로 가져옴
[root@ip-10-1-10-245 ec2-user]# docker load < busybox.tar 
50761fe126b6: Loading layer  1.442MB/1.442MB
Loaded image: busybox:latest

[root@ip-10-1-10-245 ec2-user]# docker images
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
busybox             latest              c7c37e472d31        2 weeks ago         1.22MB

** 도커 이미지 다운로드 전에 참고사항


도커 클라우드에서 dockerfile 이미지가 있는거를 확인하고 쓰는 것이 좋다.


왜냐하면 해당 도커이미지가 어떤히스토리로 어떤설정으로 만들었는지 이력을 알아야 하기 때문이다.


볼륨바인딩으로 디비데이터를 로컬로 보관이 가능하다.


#### 2. 컨테이너 데이터 백업

아래 리눅스 커맨드 명령어와 같이 컨테이너 내의 데이터를 백업할 수 있다.

In [ ]:
[root@ip-10-1-10-245 ec2-user]# pwd
/home/ec2-user

[root@ip-10-1-10-245 ec2-user]# mkdir mysqldata

[root@ip-10-1-10-245 ec2-user]# pwd
/home/ec2-user

[root@ip-10-1-10-245 ec2-user]# ls
busybox.tar  mysqldata

[root@ip-10-1-10-245 ec2-user]# cd mysqldata

[root@ip-10-1-10-245 mysqldata]# pwd
/home/ec2-user/mysqldata

mysql 컨테이너 데이터를 호스트 디렉토리 mysqldata 폴더에 저장한다.
-v옵션 : 볼륨 바인딩, 컨테이너 특정 디렉토리와 호스트 특정 디렉토리를 연결한다.
[root@ip-10-1-10-245 mysqldata]# docker run -e MYSQL_ROOT_PASSWORD='minman2115' --name mydb -d -v /home/ec2-user/mysqldata:/var/lib/mysql mysql:5.6
Unable to find image 'mysql:5.6' locally
5.6: Pulling from library/mysql
7d2977b12acb: Pull complete
5fb8400e7f07: Pull complete
234877fbb165: Pull complete
6fe1021f12f3: Pull complete
7e36fe6b53f0: Pull complete
996ec709c11b: Pull complete
5198b7523387: Pull complete
5db39dfed4be: Pull complete
2cd3a5df9c63: Pull complete
53dc4458df12: Pull complete
aebc16657535: Pull complete
Digest: sha256:19a164794d3cef15c9ac44754604fa079adb448f82d40e4b8be8381148c785fa
Status: Downloaded newer image for mysql:5.6
38c70e8cfb3181ce2a86f8a320b6f6d5ce7553bb16e419e13f00b84056038bfb

[root@ip-10-1-10-245 mysqldata]# ls
auto.cnf  ibdata1  ib_logfile0  ib_logfile1  mysql  performance_schema

mysql 컨테이너 설정을 아래와 같이 컨피그 파일도 원하는 대로 변경해서 볼륨바인딩을 걸어줄수도 있다.

In [ ]:
docker run -e MYSQL_ROOT_PASSWORD='minman2115' --name mydb -d -v /home/ec2-user/mysqldata:/var/lib/mysql -v /home/ec2-user/docker/my.cnf:/etc/mysql/conf.d/my.cnf mysql:5.6

- question : 볼륨 디렉토리 폴더 안에 이미 무언가 파일이 있다면 어케되는지? --> overwirte된다. 기존에 있는 파일도 그대로 남아있다.


-v 옵션을 하나 둬 줘서 컨피그 파일도 로컬로 보관이 가능하다. 


#### 3. 데이터 컨테이너


- 다른 컨테이너에게 볼륨을 공유하기 위해 만드는 컨테이너


- 어플리케이션을 실행하지는 않는 컨테이너


아래 리눅스 커맨드 명령어와 같이 데이터 컨테이너를 생성하고 활용할 수 있다.

In [ ]:
/home/ec2-user/mydata 디렉토리를 공유하는 데이터 컨테이너 생성
그래서 /home/ec2-user/mydata 디렉토리를 먼저 생성해준다.
[root@ip-10-1-10-245 mydata]# docker run --name mydata -v /home/ec2-user/mydata busybox true

[root@ip-10-1-10-245 mydata]# docker ps -a
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                     PORTS               NAMES
3bbe634dc558        busybox             "true"              3 seconds ago       Exited (0) 2 seconds ago                       mydata

mydata 컨테이너를 참조하는 컨테이너 생성
-it옵션 : 인터렉티브 모드
[root@ip-10-1-10-245 mydata]# docker run -it --volumes-from mydata ubuntu:14.04
Unable to find image 'ubuntu:14.04' locally
14.04: Pulling from library/ubuntu
2e6e20c8e2e6: Pull complete
30bb187ac3fc: Pull complete
b7a5bcc4a58a: Pull complete
Digest: sha256:ffc76f71dd8be8c9e222d420dc96901a07b61616689a44c7b3ef6a10b7213de4
Status: Downloaded newer image for ubuntu:14.04
root@ff2573c64243:/# ls
bin   dev  home  lib64  mnt  proc  run   srv  tmp  var
boot  etc  lib   media  opt  root  sbin  sys  usr
root@ff2573c64243:/# cd /data
bash: cd: /data: No such file or directory 
root@ff2573c64243:/# pwd
/
root@ff2573c64243:/# exit
exit

[root@ip-10-1-10-245 mydata]#